In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 https://cloud.r-project.org/bin/li

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameFunctions").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3VR960AHLFKDV|          4|            0|          0|   N|                Y|
|R16LGVMFKIUT0G|          5|            0|          0|   N|                Y|
|R1AIMEEPYHMOE4|          5|            1|          1|   N|                Y|
|R1892CCSZWZ9SR|          3|            0|          0|   N|                Y|
|R285P679YWVKD1|          3|            0|          0|   N|                N|
| RLB33HJBXHZHU|          5|            0|          0|   N|                Y|
|R1VGTZ94DBAD6A|          5|            2|          2|   N|                Y|
|R168KF82ICSOHD|          5|            0|          0|   N|                Y|
|R20DIYIJ0OCMOG|          5|            0|          0|   N|                Y|
| RD46RNVOHNZSC|          5|            0|          0|   N|     

In [8]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
vine20_df = vine_df.filter("total_votes>19")
vine20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
|R3GNSIFV1J2Y2B|          1|           15|         60|   N|                N|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|     

In [11]:
# Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
vine_helpful = vine20_df.filter(vine20_df["helpful_votes"]/vine20_df["total_votes"]>=0.5)
vine_helpful.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|                Y|
| RJNDSWES5ISZ7|          5|           78|         79|   N|     

In [13]:
# Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid)
paid = vine_helpful.filter(vine_helpful["vine"] == "Y")
paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2BQOD1R0228FN|          3|           17|         26|   Y|                N|
| RC31RUPFOHBHQ|          4|          102|        117|   Y|                N|
| REN3N1WITLF1Y|          5|           33|         37|   Y|                N|
| R71RZQ9UZVG47|          4|           38|         47|   Y|                N|
|R38NMQBH88HLM6|          4|           18|         24|   Y|                N|
|R33FGX9EE3QVR6|          4|           26|         26|   Y|                N|
|R1KIOIK6WEYE59|          3|           19|         20|   Y|                N|
|R25X9BMOB3FD0E|          4|           32|         37|   Y|                N|
|R3VCKFCX2377Q2|          4|           95|        101|   Y|                N|
|R1E0OUG63HMSM4|          3|           58|         61|   Y|     

In [14]:
# retrieve all the rows where the review was not part of the Vine program (unpaid
unpaid = vine_helpful.filter(vine_helpful["vine"] == "N")
unpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|                Y|
| RJNDSWES5ISZ7|          5|           78|         79|   N|     

In [18]:
# Count the number of five star reviews 
fivestar_review = vine20_df.filter(vine20_df["star_rating"]== 5)
fivestar_review.count()

8616

In [23]:
# Calculate the percentage of five star reviews out of the total number of reviews 
(fivestar_review.count()/vine20_df.count())*100

45.97897433160787

In [28]:
# Calculate the percentage of paid five star reviews 
fivestar_paid= paid.filter(paid["star_rating"]== 5)
(fivestar_paid.count()/paid.count())*100

54.41176470588235

In [29]:
# Calculate the percentage of unpaid five star reviews 
fivestar_unpaid= unpaid.filter(unpaid["star_rating"]== 5)
(fivestar_unpaid.count()/unpaid.count())*100

47.072534546867196

In [30]:
paid.count()

136

In [31]:
unpaid.count()

18019

In [32]:
fivestar_paid.count()

74

In [33]:
fivestar_unpaid.count()

8482